# Sum Frame Sudoku puzzle

Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.
Numbers in the outside frame equal the sum of the first three numbers in the corresponding row or column
in the given direction.

Find more examples [here](http://frame-sudoku.blogspot.com)

Imports

In [1]:
from ortools.sat.python import cp_model
from itertools import product
import numpy as np

Sudoku puzzle from slides

In [2]:
top = [15, 18, 12, 11, 21, 13, 15, 17, 13]
right = [22, 8, 15, 22, 12, 11, 15, 13, 17]
bottom = [15, 9, 21, 10, 16, 19, 13, 15, 17]
left = [8, 15, 22, 11, 13, 21, 18, 19, 8]
n = 9

And another example (commented out)

In [3]:
# top = [21, 12, 12, 13, 14, 18, 10, 19, 16]
# right = [20, 15, 10, 22, 8, 15, 17, 15, 13]
# bottom = [17, 9, 19, 18, 13, 14, 23, 15, 7]
# left = [12, 12, 21, 14, 14, 17, 14, 9, 22]

Create model

In [4]:
model = cp_model.CpModel()

In [2]:
# create board
board = [[model.NewIntVar(1, n, f"({j})") for j in range(1, n+1)] for i in range(1,n+1)]

# the numbers occur only once in each row, each column or 3x3 square
for i in range(n):
    model.AddAllDifferent(board[i]) # rows
    model.AddAllDifferent([board[j][i] for j in range(n)]) # columns
    
# Each 3x3 sub-grid contains all numbers from 1 to 9 once
for i in range(3):
    for j in range(3): 
        model.AddAllDifferent([board[i * 3 + di][j* 3 + dj] for di in range(3) for dj in range(3)])

        
for i in range(n):
    model.Add(cp_model.LinearExpr.Sum([board[j][i] for j in range(3)]) == top[i]) # top sums
    model.Add(cp_model.LinearExpr.Sum([board[i][j] for j in range(3)]) == left[i]) # left sums
    model.Add(cp_model.LinearExpr.Sum([board[6][i],board[7][i],board[8][i]]) == bottom[i]) # bottom sums
    model.Add(cp_model.LinearExpr.Sum([board[i][6],board[i][7],board[i][8]]) == right[i]) # right sums
    


NameError: name 'n' is not defined

Callback for solution printing (adapt if you do not use an n*n board)

In [6]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        
    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{self.Value(self.__variables[i][j])}] ", end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [7]:
solver = cp_model.CpSolver()
_ = solver.SearchForAllSolutions(model, SolutionPrinter(board))

[1] [4] [3] [2] [8] [5] [9] [6] [7] 

[8] [5] [2] [6] [9] [7] [4] [3] [1] 

[6] [9] [7] [3] [4] [1] [2] [8] [5] 

[2] [3] [6] [7] [1] [4] [8] [5] [9] 

[4] [8] [1] [9] [5] [6] [3] [7] [2] 

[9] [7] [5] [8] [2] [3] [6] [1] [4] 

[7] [2] [9] [1] [3] [8] [5] [4] [6] 

[5] [6] [8] [4] [7] [2] [1] [9] [3] 

[3] [1] [4] [5] [6] [9] [7] [2] [8] 






In [8]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")

Runtime:   0.025222ms
Booleans:  26
Failures:  99
Branches:  176
